<a href="https://colab.research.google.com/github/PBS-Alekhya/SpamDataCollection/blob/main/ModelBuilding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install emoji

Text cleaning before normalization

In [33]:
import re
import emoji

def clean_text(text):
    text = text.lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '<URL>', text)       # Replace URLs
    text = re.sub(r'[^\w\s<>]', '', text)                          # Remove punctuation
    text = re.sub(r'\b[a-zA-Z]*\d+[a-zA-Z]*\b', '', text)          # Remove alphanumeric codes
    text = re.sub(r'\b\d+\b', '', text)                            # Remove standalone numbers
    text = re.sub(r'\s+', ' ', text).strip()                       # Normalize whitespace
    return text

In [34]:
import pandas as pd

df = pd.read_csv("dataset.csv")
df['post'] = df['post'].apply(clean_text)

df.to_csv("cleaned_dataset.csv", index=False)

In [35]:
df.tail(20)

,post,Label
3470,reminder your lic premium payment is due tomor...,not_spam
3471,all the best for your gate exam tomorrow,not_spam
3472,exclusive deal on your mobile recharge visit <...,spam
3473,act now flipkart big sale ends soon shop here ...,spam
3474,some newcomers will complain get angry or even...,not_spam
3475,preapproved for a loan apply instantly,spam
3476,in the broader market the nse midcap and small...,not_spam
3477,act now flipkart big sale ends soon shop here ...,spam
3478,hi siddhartha i have updated sheet for work fr...,not_spam
3479,the federal reserve has access to even more da...,not_spam


it is hinglish dictionary

In [36]:
NORMALIZATION_DICT = {
    # Greetings & People
    "hello": "hello",       "hi": "hi",               "namaste": "hello",
    "bhai": "bro",          "dost": "friend",

    # Offers & Actions
    "jeeto": "win",         "kamaye": "earn",         "kamana": "earn",
    "paye": "get",          "le": "take",             "leja": "take",
    "leliya": "took",       "click": "click",         "bina": "without",
    "signup": "signup",     "register": "register",   "join": "join",
    "free": "free",         "paise": "money",         "paisa": "money",
    "offer": "offer",       "fatafat": "quick",       "abhi": "now",
    "turant": "immediately","instant": "instant",     "lo": "take",
    "mil": "get",           "milta": "get",           "milgaya": "got",
    "bhejo": "send",        "bhej": "send",           "pao": "get",
    "paoge": "get",

    # Rewards & Currency
    "recharge": "recharge", "cashback": "cashback",   "gift": "gift",
    "voucher": "voucher",   "prize": "prize",         "rewards": "rewards",
    "points": "points",     "lakh": "lakh",           "crore": "crore",
    "rupees": "rupees",     "rs": "rupees",           "rs.": "rupees",
    "inr": "rupees",

    # Platforms & Tech
    "whatsapp": "whatsapp", "sms": "sms",             "msg": "message",
    "message": "message",   "link": "link",           "app": "app",
    "mobile": "mobile",     "number": "number",       "phone": "phone",

    # Common Words
    "kya": "what",          "hai": "is",              "hoga": "will be",
    "kar": "do",            "karo": "do",             "karte": "doing",
    "ab": "now",            "bahut": "very",          "zyada": "very",
    "accha": "good",        "achha": "good",          "mast": "great",
    "bindaas": "cool",      "best": "best",

    # Urgency & Security
    "urgent": "urgent",     "important": "important", "last": "last",
    "chance": "chance",     "limited": "limited",     "alert": "alert",
    "verify": "verify",     "otp": "otp",             "account": "account",
    "block": "block",       "blocked": "blocked"
}



Normalization of the hinglish terms in posts

In [60]:
import pandas as pd

data = pd.read_csv("cleaned_dataset.csv")

data.shape

(3490, 2)

In [38]:
import re

def normalization(post, hinglish_dict):
  if not isinstance(post, str):
        post = str(post)
  words = post.split()
  normalized_words = [hinglish_dict.get(word, word) for word in words]
  return " ".join(normalized_words)


In [39]:
data["normalized_text"] = data["post"].apply(
    lambda x: normalization(x, NORMALIZATION_DICT)
)


In [40]:
data.to_csv("normalized_dataset.csv", index=False)


In [58]:
data.shape

(3490, 3)

The next step is to create a vectorizer using tf-idf

In [41]:
HINGLISH_STOP_WORDS = list(set([
    # Common Hinglish & Hindi functional words
    "hai", "ka", "ki", "ke", "ko", "se", "mein", "par", "tha", "thi", "hai", "tha",
    "nahi", "na", "bhi", "ye", "wo", "kya", "ho", "raha", "hoga", "kar", "karo", "hua",
    "ab", "jab", "sab", "tum", "aap", "hum", "unka", "suno", "unki", "tumki", "mera", "meri", "mere",

    # Common English stop words
    "i", "me", "my", "myself", "we", "our", "ours", "ourselves",
    "you", "your", "yours", "yourself", "yourselves",
    "he", "him", "his", "himself", "she", "her", "hers", "herself",
    "it", "its", "itself", "they", "them", "their", "theirs", "themselves",
    "what", "which", "who", "whom", "this", "that", "these", "those",
    "am", "is", "are", "was", "were", "be", "been", "being",
    "have", "has", "had", "having", "do", "does", "did", "doing",
    "a", "an", "the", "and", "but", "if", "or", "because", "as",
    "until", "while", "of", "at", "by", "for", "with", "about", "against",
    "between", "into", "through", "during", "before", "after", "above", "below",
    "to", "from", "up", "down", "in", "out", "on", "off", "over", "under",
    "again", "further", "then", "once", "here", "there", "when", "where", "why",
    "how", "all", "any", "both", "each", "few", "more", "most", "other", "some",
    "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very",
    "can", "will", "just", "don", "should", "now", "which", "though", "should"
]))


In [81]:
import pandas as pd
df = pd.read_csv("normalized_dataset.csv")
df = df.dropna(subset=['post', 'Label','normalized_text'])
corpus = df["normalized_text"]



(3460,)

In [82]:
from sklearn.feature_extraction.text import TfidfVectorizer

# vectorizer = TfidfVectorizer(max_features=5000)  # Adjust max_features if needed
#

vectorizer = TfidfVectorizer(
    stop_words=HINGLISH_STOP_WORDS,
    max_features=5000,
    token_pattern=r'(?u)\b[a-z]{3,15}\b'  # Only alphabetic words, 3–15 chars
)
X = vectorizer.fit_transform(corpus)



In [84]:
print(vectorizer.get_feature_names_out()[:20])  # See first 20 words


['aadhaar' 'aagey' 'aaj' 'aanandam' 'aanu' 'aati' 'aayrikkum' 'abhishek'
 'able' 'abp' 'abroad' 'acc' 'access' 'accessories' 'according'
 'accordingly' 'account' 'accountadd' 'accounts' 'accumulate']


In [85]:
X.shape

(3460, 3462)

In [93]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(df["Label"])  # 'spam' → 1, 'not_spam' → 0
y.shape

(3460,)

Model Building using naives Bayes and Logicstic Regression

In [94]:
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Choose model
model = LogisticRegression()  # or MultinomialNB()

# Train
model.fit(X_train, y_train)


LogisticRegression()

In [95]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=le.classes_))


              precision    recall  f1-score   support

    not_spam       0.95      1.00      0.98       334
        spam       1.00      0.95      0.98       358

    accuracy                           0.98       692
   macro avg       0.98      0.98      0.98       692
weighted avg       0.98      0.98      0.98       692



In [91]:
from sklearn.naive_bayes import MultinomialNB

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Choose model
model =  MultinomialNB()

# Train
model.fit(X_train, y_train)


MultinomialNB()

In [92]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=le.classes_))

              precision    recall  f1-score   support

    not_spam       0.98      0.94      0.96       334
        spam       0.95      0.98      0.96       358

    accuracy                           0.96       692
   macro avg       0.96      0.96      0.96       692
weighted avg       0.96      0.96      0.96       692



Generating .pkl files


In [96]:
import joblib

# Save the Logistic Regression model
joblib.dump(model, 'LRmodel.pkl')

# Save the TF-IDF vectorizer
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')

print("Logistic Regression model and vectorizer have been saved successfully.")


Logistic Regression model and vectorizer have been saved successfully.
